In [ ]:
#pip install yfinance

In [ ]:
#pip install pandas-datareader

In [ ]:
#pip install plotly

In [2]:
import pandas as pd # tool for data processing
from pandas_datareader import data # tool for data processing
import plotly.graph_objects as go # tool for data visualization
import yfinance as yf # tool for downloading histrocial market data from "Yahoo! Finance"
from datetime import date # tool for manipulating dates and times
from dateutil.relativedelta import relativedelta # tool for manipulating dates and times
import numpy as np # tool for handling vectors, matrices or large multidimensional arrays
from sklearn.linear_model import LinearRegression # tool for machine leraning (Linear Model)
from sklearn.model_selection import train_test_split # tool for machine learning
import matplotlib.pyplot as plt
import seaborn as sns
#import riskfolio as rp
import plotly.express as px

In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [4]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

#

# creating list for stock indexes to select from
stock_indices = ['^GSPC','^DJI', '^IXIC','^NYA','^RUT']


#creating function to check the invalid ticker which does not exist in yahoo finance
def is_valid_ticker(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    return len(ticker.history(period='1d')) > 0

#Using while-True to prevent wrong user inputs:

while True:
    try:
        # Selecting list to save the selected stocks
        selected_stocks = []
        while not selected_stocks:
            selected_stocks = input("Please enter the ticker symbols (comma-separated) for the stocks you want to include in your portfolio:\n").split(',')
            selected_stocks = [s.strip() for s in selected_stocks]

        # Calculate the number of assets selected
        num_assets = len(selected_stocks)

        # Display the number of assets selected
        print(f"{num_assets} asset(s) selected")

        # Check if the selected tickers are valid
        invalid_tickers = []
        for ticker in selected_stocks:
            if not is_valid_ticker(ticker):
                invalid_tickers.append(ticker)

        # Print the invalid tickers
        if invalid_tickers:
            print("Invalid tickers:")
            for ticker in invalid_tickers:
                print(ticker)

            # Prompt the user to enter tickers again
            print("Please enter the tickers again:")
            continue

        # Select the start and end date
        sdate = input('Please enter the start date (YYYY-MM-DD) for your portfolio:\n')
        edate = input('Please enter the end date (YYYY-MM-DD) for your portfolio:\n')

        # Ask the user to select an index from the list
        print("Select a stock index:")
        for i, index in enumerate(stock_indices):
            print(f"{i+1}: {index}")
        selection = int(input("Enter your selection: "))

        # Now the user can enter their preferred date range (sdate = start date; edate = end date)
        stock_data = yf.download(selected_stocks, start=sdate, end=edate)['Adj Close'].dropna()
        stock_data = pd.DataFrame(stock_data)
        index_data = yf.download(stock_indices[selection-1], start=sdate, end=edate)['Adj Close'].dropna()

        break  # Exit the loop once the assets are selected

    except (KeyError, OSError):
        print(color.BOLD + color.UNDERLINE + f'> Invalid ticker(s) entered. Please try again...' + color.END)

        
        
#declaring some variables and saving the results which will be called in multiple options        
# Ask the user for the amount of money to be invested
amount = float(input("Enter the amount of money to be invested: "))
#creating variable for equal weights
equal_weight = 1 / len(selected_stocks)
# Ask the user whether to assign weights manually or use equal weights
choice = input("Do you want to assign weights manually? (Yes/No): ")
if choice.lower() == "yes":
    # Ask the user to enter the weights for each stock
    while True:
        weights = []
        for stock in selected_stocks:
            weight = float(input(f"Enter weight for {stock} in decimal (e.g 0.2, 0.3): "))
            weights.append(weight)

        # Normalize the weights to add up to 1
        total_weight = sum(weights)
        if abs(total_weight - 1) < 1e-6:
            break
        else:
            print("Error: The sum of weights must be equal to 1. Please try again.")
else:
    # Assign equal weights to all stocks
    weights = [equal_weight] * len(selected_stocks)
# Calculate the amount to be invested in each stock
invested_money = [amount * weight for weight in weights]

# Calculate daily portfolio values
daily_portfolio_values = (stock_data * weights).sum(axis=1)

# Calculate total invested money
total_invested_money = amount * sum(weights)

# Filter the daily portfolio values by the selected date range
portfolio_values = daily_portfolio_values.loc[sdate:edate]

# Calculate the total return of the portfolio
total_return = (portfolio_values.iloc[-1] - portfolio_values.iloc[0]) / portfolio_values.iloc[0]

# Calculate the total value of the portfolio if $X was invested
portfolio_value = (total_return + 1) * amount


#show features of the tool        
print(color.BOLD + color.UNDERLINE + '\n> Have a look at the different options on analyzing the stock data:' + color.END)
choice = True
while choice:
    print(f'''\nWhat would you like to do: 
    A - Show me the Assset Comparison
    B - Show me Portfolio Performance
    C - Show me the Forecast on Portfolio Return
    Q - Quit the program
    ''')
    choice = (input('''\nPlease enter your choice (e.g. for the first option type "A"): '''))
    
    

    #Setting condition to explore the multiple features of the portfolio analysis tool
    # OPTION 1: Show me the price chart of my chosen stock
    
    if choice == 'A' or choice == 'a':
        # -----------Download the stock data--------------
        stock_data = yf.download(selected_stocks, start=sdate, end=edate)['Adj Close'].dropna()
        stock_data.head()

        # Create traces for each stock
        traces = []
        for stock in stock_data.columns:
            traces.append(go.Scatter(
                x=stock_data.index,
                y=stock_data[stock],
                mode='lines',
                name=stock
            ))

        # Create the layout for the plot
        layout = go.Layout(
            title=f'Stock Prices from {sdate} to {edate}',
            xaxis=dict(title='Date'),
            yaxis=dict(title='Adjusted Closing Price in USD'),
            xaxis_tickfont_size=14,
            yaxis_tickfont_size=14
        )

        # Create a line chart using Plotly
        fig = go.Figure(data=traces, layout=layout)

        # Display the plot
        fig.show()

        #--------------Let's calculate the return of the stocks----------------------    
        returns = stock_data.pct_change()
        returns.dropna(inplace=True)
        #calculate percetange return for each stock
        pct_returns = returns*100

        #calculating reuten of the stock index
        index_return = index_data.pct_change().dropna()
        pct_index_return = index_return.dropna()*100

        #show percentage return of each stock from the sdate to edate
        pct_change = (stock_data.iloc[-1] / stock_data.iloc[0] - 1) * 100
        print(f"Asset Returns in Portfolio from {sdate} to {edate} is:\n\n", 
        "\n".join([f"{stock}: {pct_change:.2f}%" for stock, pct_change in pct_change.items()]))


        #--------------------Show correlation between the assets in the portfolio-----------------
        corr_mat = returns.corr().round(2)
        print(corr_mat)

        
        #-----Calculate volatility for each stock-------
        # Here we are using a rolling() function to determine the standard deviation of returns as the data is time series
        # the optimal window for rolling std is 20-30 trading days
        rolling_volatility = pct_returns.rolling(window=30).std() * (30**0.5)
        rolling_volatility.dropna(inplace=True)

        # Plot the volatility of each stock using Plotly
        fig = go.Figure()
        for stock in rolling_volatility.columns:
            fig.add_trace(go.Scatter(x=rolling_volatility.index, y=rolling_volatility[stock], mode='lines', name=stock, hovertemplate='<br>Date: %{x}<br>'+ stock + ': %{y:.2f}%<extra></extra>'))

        fig.update_layout(title="Volatility Comparison",
                              xaxis_title='Date',
                              yaxis_title='% Volatility')

        fig.show()


        #---------------------------calculate the drawdown for each stock---------------------------
        # Calculate the running maximum of the daily stock price
        running_maximum = stock_data.rolling(30, min_periods=1).max()
        # Calculate the drawdown as the percentage decline from the running maximum
        daily_drawdown = stock_data/running_maximum - 1.0
        # Calculate the rolling maximum drawdown 
        max_daily_drawdown = daily_drawdown.rolling(30, min_periods=1).min() * 100
        # Drop any rows with missing values
        max_daily_drawdown.dropna(inplace=True)

        # Plot the daily_drawdown of each stock using Plotly
        fig = go.Figure()
        for stock in max_daily_drawdown.columns:
            fig.add_trace(go.Scatter(x=max_daily_drawdown.index, y=max_daily_drawdown[stock], mode='lines', name=stock, hovertemplate='<br>Date: %{x}<br>'+ stock + ': %{y:.2f}%<extra></extra>'))

        fig.update_layout(title="Drawdown Comparison",
                              xaxis_title='Date',
                              yaxis_title='% Drawdown')

        fig.show()



        
    # OPTION 2: Show me analysis on my portfolio's return
    elif choice == 'B' or choice == 'b':

        # Create a pandas dataframe with the weights and invested money for each stock
        w = pd.DataFrame(list(zip(weights, invested_money)), index=selected_stocks, columns=["Weight", "Invested Money"])

        # Normalize the weights to add up to 1
        total_weight = sum(weights)
        weights = [weight / total_weight for weight in weights]

        # Print the weights assigned to each stock
        print("\nAsset Allocation\n")
        for i in range(len(selected_stocks)):
          print(f"{selected_stocks[i]}: {weights[i]:.2%} ({invested_money[i]:.2f} USD)")

        # Plotting Asset Allocation pie-plot
        #rp.plot_pie(pd.DataFrame(w["Weight"]), title='Asset Allocation')

        # Calculate the total return of the portfolio
        #total_return = (portfolio_values.iloc[-1] - portfolio_values.iloc[0]) / portfolio_values.iloc[0]

        # Calculate the total value of the portfolio if $X was invested
        #portfolio_value = (total_return + 1) * amount

        # Calculate the percentage change in the portfolio values since start date
        start_value = portfolio_values.iloc[0]
        portfolio_pct_change = ((portfolio_values - start_value) / start_value) * 100

        # Calculate the net profit or loss in dollars and percentage
        net_profit_loss = portfolio_value - total_invested_money
        net_profit_loss_pct = net_profit_loss / total_invested_money

        # Display the results to the user
        print(f"\nYour investment of ${amount:.2f} on {sdate} be worth ${portfolio_value:.2f} by {edate} in your Portfolio.")
        if net_profit_loss > 0:
            print(f"You would have made a profit of ${net_profit_loss:.2f} ({net_profit_loss_pct:.2%}).")
        else:
            print(f"You would have incurred a loss of ${abs(net_profit_loss):.2f} ({abs(net_profit_loss_pct):.2%}).")

        # Plot the portfolio returns using Plotly
        import plotly.graph_objects as go
        returns_trace = go.Scatter(
            x=portfolio_values.index,
            y=portfolio_values.values,
            mode='lines',
            name='Portfolio Returns',
            hovertemplate='<br>Date: %{x}<br>Portfolio Value: $%{y:.2f} (%{text:.2f}%)',
            text=portfolio_pct_change
        )
        # Create a layout for the plot
        layout = go.Layout(
            title='Portfolio Performance',
            xaxis=dict(title='Date'),
            yaxis=dict(title='Portfolio Value')
        )

        # Create the figure and add the trace and layout
        fig = go.Figure(data=[returns_trace], layout=layout)

        # Show the plot

        fig.show()
        
        
        #----------Calculate the daily returns for the portfolio and benchmark---------------
        portfolio_returns = daily_portfolio_values.pct_change().dropna()
        benchmark_returns = index_data.pct_change().dropna()

        # Calculate the cumulative returns for the portfolio and benchmark
        portfolio_cum_returns = (1 + portfolio_returns).cumprod()
        benchmark_cum_returns = (1 + benchmark_returns).cumprod()

        # Plot the portfolio and benchmark cumulative returns using Plotly
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=portfolio_cum_returns.index, y=portfolio_cum_returns.values, name='Portfolio',
                                 hovertemplate='Portfolio: $%{y:.2f} (%{text}%)<br>%{x}<extra></extra>',
                                 text=[f'{p:.2f}' for p in portfolio_returns[-len(portfolio_cum_returns):] * 100]))
        fig.add_trace(go.Scatter(x=benchmark_cum_returns.index, y=benchmark_cum_returns.values, name='Benchmark',
                                 hovertemplate='Benchmark: $%{y:.2f} (%{text}%)<br>%{x}<extra></extra>',
                                 text=[f'{b:.2f}' for b in benchmark_returns[-len(benchmark_cum_returns):] * 100]))
        fig.update_layout(title='Portfolio Vs Benchmark Performance',
                          xaxis_title='Date',
                          yaxis_title='Cumulative Return')

        fig.show()
        
        
        #-----------------Calculate Sharpe Ratio----------------------------
        #using FRED API key to read the T-bill rate
        from fredapi import Fred
        fred = Fred(api_key='78b14ec6ba46f484b94db43694468bb1')
        #get three-month treasury bill rate for the selected date range
        risk_free = fred.get_series('DGS3MO')[sdate:edate]
        #rf-rate is in percentage thats why needs to be divided by 100
        #also we are taking yearly trading days to get the daily rf for the selected date range span
        daily_rf = (risk_free/100)/252
        daily_stock_returns = stock_data.pct_change()
        daily_stock_returns.dropna(inplace=True)
        
        #calculating excess return of the stocks
        for i in selected_stocks:
            i = i.upper()
            daily_stock_returns[f"excess_return_{i}"] = daily_stock_returns[i] - daily_rf
        #filtering only excess_return columns    
        excess_returns = daily_stock_returns.filter(like='excess_return_')
        # Define the rolling window for the Sharpe Ratio
        rolling_window = 52 # 2 months x 21 trading days/month

        # Calculate the rolling Sharpe Ratio
        rolling_mean = excess_returns.rolling(rolling_window).mean()
        rolling_std = excess_returns.rolling(rolling_window).std()
        rolling_sharpe_ratio = (rolling_mean / rolling_std) * (252**0.5)
        rolling_sharpe_ratio.columns = rolling_sharpe_ratio.columns.str.replace('excess_return_', '') + '_sharpe_ratio'
        rolling_sharpe_ratio.dropna(inplace=True)
        
        #plotting using plotly
        fig = go.Figure()
        for stock in rolling_sharpe_ratio.columns:
            fig.add_trace(
                go.Scatter(x=rolling_sharpe_ratio.index, 
                           y=rolling_sharpe_ratio[stock], 
                           mode='lines', 
                           name=stock, 
                           hovertemplate='<br>Date: %{x}<br>'+ stock + ': %{y:.2f}<extra></extra>')
            )

        fig.update_layout(title="Sharpe Ratio Comparison",
                          xaxis_title='Date',
                          yaxis_title='Sharpe Ratio')

        fig.show()
        
        
        #-------------See risk return tradeoff----------------------
        #Converting the excess return and volatility in percentage
        mean_return = excess_returns.mean()*100
        volatility = excess_returns.std()*100

        #Create a scatter plot
        plt.scatter(volatility, mean_return, s=50, c='blue', edgecolors='black')

        #Add labels to the data points
        for i, txt in enumerate(excess_returns.columns.tolist()):
          plt.annotate(txt.split('_')[-1], (volatility[i], mean_return[i]), ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')

        #Set the labels and title
        plt.xlabel('Volatility (%)')
        plt.ylabel('Mean Excess Return (%)')
        plt.title('Risk-Return Tradeoff')

        #Add the average line originating from the origin
        x = np.linspace(0, max(volatility), 100)
        y = x * np.mean(mean_return) / np.mean(volatility)
        plt.plot(x, y, color='red', linestyle='dashed')

        #Display the plot
        plt.show()        
        

        
        

    # OPTION 3: Show me forecast on my Portfolio's Performance
    elif choice == 'C' or choice == 'c':
        
        #taking first difference to get feed it pacf for calculating optimal lag
        portfolio_diff = portfolio_values.diff().dropna()
        
        import statsmodels.api as sm
        
        # Calculate the PACF values
        pacf_values = sm.tsa.pacf(portfolio_values, nlags=25)

        # Calculate the AIC values for different lag lengths
        aic_values = []
        for lag in range(1, len(pacf_values)):
            model = sm.tsa.AutoReg(portfolio_values, lags=lag).fit()
            aic_values.append(model.aic)

        # Find the lag length with the minimum AIC value
        optimal_lag = aic_values.index(min(aic_values)) + 1

        # Find the next significant lag after lag 1
        significant_lags = []
        for lag, pacf in enumerate(pacf_values[optimal_lag:], start=optimal_lag):
            if abs(pacf) >= 2 / (len(portfolio_values) ** 0.5):
                significant_lags.append(lag)
                break  # Stop searching after finding the first significant lag
         
        #set train and test length of the dataset
        from statsmodels.tsa.arima.model import ARIMA
        train_len = int(len(portfolio_values)*0.8)
        train = portfolio_values[:train_len]
        test = portfolio_values[train_len:]

        #give start and end point
        start = len(train)
        end = len(train) + len(test) - 1

        #fit data in arima model
        if len(significant_lags) > 0 and significant_lags[0] != 0:
            arima_model = ARIMA(train, order=(significant_lags[0], 1, significant_lags[0]))
        else:
            arima_model = ARIMA(train, order=(8, 1, 8))  # Use default order (8, 1, 8) if no significant lag

        arima_result = arima_model.fit()

        # Predict ARIMA model
        arima_predict = arima_result.predict(start, end)
        arima_predict.index = test.index

        #mse score
        from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
        mape_arima = mean_absolute_percentage_error(test, arima_predict)
        mae_arima = mean_absolute_error(test, arima_predict)
        mse_arima = mean_squared_error(test, arima_predict)
        rmse_arima = np.sqrt(mean_squared_error(test, arima_predict))
        arima_perf = {'mape_arima':mape_arima,
                    'mae_arima':mae_arima,
                    'mse_arima':mse_arima,
                    'rmse_arima':rmse_arima}
        arima_perf = pd.DataFrame([arima_perf])

        # Forecast using ARIMA
        if len(significant_lags) > 0 and significant_lags[0] != 0:
            arima_model = ARIMA(portfolio_values, order=(significant_lags[0], 1, significant_lags[0]))
        else:
            arima_model = ARIMA(portfolio_values, order=(8, 1, 8))  # Use default order (8, 1, 8) if no significant lag

        arima_result = arima_model.fit()

        # Define the forecast index
        forecast_index = pd.date_range(start=portfolio_values.index[-1], periods=30, freq='D')
        arima_forecast = arima_result.forecast(steps=30)
        arima_forecast.index = forecast_index

        #plot the forecasted result
        import plotly.graph_objs as go
        # Create traces
        trace_actual = go.Scatter(x=portfolio_values.index, y=portfolio_values.values, name='Actual')
        trace_forecast = go.Scatter(x=arima_forecast.index, y=arima_forecast.values, name='ARIMA Forecast')
        # Define layout
        layout = go.Layout(title='ARIMA forecast', xaxis=dict(title='Date'), yaxis=dict(title='Portfolio Values'))
        # Create figure
        fig = go.Figure(data=[trace_actual, trace_forecast], layout=layout)
        # Show figure
        fig.show()

        #------------------Calculate the net profit or loss in dollars and percentage-----------------
        forecasted_return = (arima_forecast[-1] - portfolio_values.iloc[-1]) / portfolio_values.iloc[-1]
        forecasted_value = (forecasted_return + 1) * portfolio_value
        net_profit_loss = forecasted_value - portfolio_value
        net_profit_loss_pct = net_profit_loss / portfolio_value

        # Display the results to the user
        if net_profit_loss > 0:
            print(f"You would make profit of ${net_profit_loss:.2f} ({net_profit_loss_pct:.2%}) over the forecasted period.")
        else:
            print(f"You would incurr a loss of ${abs(net_profit_loss):.2f} ({abs(net_profit_loss_pct):.2%}) over the forecasted period.")

        print('\n',arima_perf)

    elif choice == 'Q' or choice == 'q':
        print(color.YELLOW + color.BOLD + '\n> Thank you for using our Portfolio Analysis Tool :)' + color.END)
        choice = None
        
    #for invalid user input choice    
    else:
        print(color.BOLD + color.UNDERLINE + '\n> Your input is not a given choice. Please try again...' + color.END) 

3 asset(s) selected
Select a stock index:
1: ^GSPC
2: ^DJI
3: ^IXIC
4: ^NYA
5: ^RUT
[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  1 of 1 completed

> Have a look at the different options on analyzing the stock data:

What would you like to do: 
    A - Show me the Assset Comparison
    B - Show me Portfolio Performance
    C - Show me the Forecast on Portfolio Return
    Q - Quit the program
    


c:\Users\abc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\abc\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
c:\Users\abc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\abc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and s

You would make profit of $37.07 (2.43%) over the forecasted period.

    mape_arima  mae_arima   mse_arima  rmse_arima
0    0.131369  27.992754  936.865499   30.608259

What would you like to do: 
    A - Show me the Assset Comparison
    B - Show me Portfolio Performance
    C - Show me the Forecast on Portfolio Return
    Q - Quit the program
    

> Thank you for using our Portfolio Analysis Tool :)


## LSTM Implementation

In [8]:
#importing necessary library
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

#set train and test length of the dataset
train_len = int(len(portfolio_values) * 0.8)
train = portfolio_values[:train_len]
test = portfolio_values[train_len:]

# Normalize the data for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
test_scaled = scaler.transform(test.values.reshape(-1, 1))

# Creating sequences for LSTM
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequence = data[i : (i + seq_length)]
        sequences.append(sequence)
    return np.array(sequences)

#Adjusting squence length
seq_length = 10  
X_train = create_sequences(train_scaled, seq_length)
y_train = train_scaled[seq_length:]
X_test = create_sequences(test_scaled, seq_length)

# Reshaping data for LSTM input
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=30, input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=2))
model.compile(optimizer='adam', loss='mean_squared_error') #selecting ADAM optimizer for its robustsness

# Define early stopping callback to prevent overfitting
es_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the LSTM model
history = model.fit(
    X_train_reshaped,
    y_train,
    epochs=25,
    batch_size=20,
    validation_split=0.1,
    callbacks=[es_callback]
)

# Make predictions on the test set with LSTM
predictions_scaled_lstm = model.predict(X_test_reshaped)
predictions_lstm = scaler.inverse_transform(predictions_scaled_lstm)

Epoch 1/25
9/9 [==============================] - 2s 54ms/step - loss: 0.3110 - val_loss: 0.4112
Epoch 2/25
9/9 [==============================] - 0s 7ms/step - loss: 0.1361 - val_loss: 0.1359
Epoch 3/25
9/9 [==============================] - 0s 7ms/step - loss: 0.0344 - val_loss: 0.0100
Epoch 4/25
9/9 [==============================] - 0s 7ms/step - loss: 0.0214 - val_loss: 0.0128
Epoch 5/25
9/9 [==============================] - 0s 7ms/step - loss: 0.0192 - val_loss: 0.0065
Epoch 6/25
9/9 [==============================] - 0s 6ms/step - loss: 0.0122 - val_loss: 0.0114
Epoch 7/25
9/9 [==============================] - 0s 6ms/step - loss: 0.0118 - val_loss: 0.0090
Epoch 8/25
9/9 [==============================] - 0s 6ms/step - loss: 0.0096 - val_loss: 0.0061
Epoch 9/25
9/9 [==============================] - 0s 6ms/step - loss: 0.0088 - val_loss: 0.0064
Epoch 10/25
9/9 [==============================] - 0s 6ms/step - loss: 0.0079 - val_loss: 0.0063
Epoch 11/25
9/9 [=====================

In [14]:
# Assume new variables 
actual_test_values = test 
lstm_predictions = predictions_lstm

# Reshape predictions for comparison 
lstm_predictions_reshaped = lstm_predictions.reshape(-1, 1)

# Adjust lengths
lstm_predictions_reshaped = lstm_predictions_reshaped[:len(actual_test_values)]

# Just like we calculated performance metrics for ARIMA let calculate metrics for LSTM
mape_lstm = mean_absolute_percentage_error(actual_test_values[:len(lstm_predictions_reshaped)], lstm_predictions_reshaped)
mae_lstm = mean_absolute_error(actual_test_values[:len(lstm_predictions_reshaped)], lstm_predictions_reshaped)
mse_lstm = mean_squared_error(actual_test_values[:len(lstm_predictions_reshaped)], lstm_predictions_reshaped)  
rmse_lstm = np.sqrt(mean_squared_error(actual_test_values[:len(lstm_predictions_reshaped)], lstm_predictions_reshaped))
# Dictionary of metrics  
lstm_perf = {'mape_lstm': mape_lstm, 
             'mae_lstm': mae_lstm,
             'mse_lstm': mse_lstm,
             'rmse_lstm': rmse_lstm}

# Convert to dataframe
lstm_perf = pd.DataFrame([lstm_perf])


In [15]:
#check lstm's performance
lstm_perf

,mape_lstm,mae_lstm,mse_lstm,rmse_lstm
0,0.056675,12.158788,210.089387,14.494461


In [16]:
#check arima's performance
arima_perf

,mape_arima,mae_arima,mse_arima,rmse_arima
0,0.131369,27.992754,936.865499,30.608259


Here we can see that LSTM outperformed ARIMA model.

## For Future Work

### Using Prophet

In [ ]:
from prophet import Prophet

# Prepare the data
df = pd.DataFrame({'ds': portfolio_values.index, 'y': portfolio_values.values})
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

# Initialize the model
m = Prophet()

# Fit the model to the training data
m.fit(train_df)

# Make forecasts for the next 60 days
future = m.make_future_dataframe(periods=5)
forecast = m.predict(future)

# Plot the forecasts
fig = m.plot(forecast)
plt.title('Prophet Forecast')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.show()

# Calculate MAE and MSE
y_true = test_df['y'].values
y_pred = forecast.tail(len(test_df))['yhat'].values
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")

In [ ]:
# Calculate the net profit or loss in dollars and percentage
forecasted_return = (forecast['yhat'].iloc[-1] - portfolio_values.iloc[-1]) / portfolio_values.iloc[-1]
forecasted_value = (forecasted_return + 1) * portfolio_value
net_profit_loss = forecasted_value - portfolio_value
net_profit_loss_pct = net_profit_loss / portfolio_value

# Display the results to the user
if net_profit_loss > 0:
    print(f"You would make a profit of ${net_profit_loss:.2f} ({net_profit_loss_pct:.2%}) over the forecasted period.")
else:
    print(f"You would incurr a loss of ${abs(net_profit_loss):.2f} ({abs(net_profit_loss_pct):.2%}) over the forecasted period.")


# FOR OPTIMIZATION

Optimization for calculating the optimal weights for the assets in portfolio.

In [ ]:
# Creating the Portfolio Object
port = rp.Portfolio(returns=returns)

# To display dataframes values in percentage format
pd.options.display.float_format = '{:.4%}'.format

# Choose the risk measure
rm = 'MSV'  # Semi Standard Deviation

# Estimate inputs of the model (historical estimates)
method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Estimate the portfolio that maximizes the risk adjusted return ratio
w1 = port.optimization(model='Classic', rm=rm, obj='Sharpe', rf=0.0, l=0, hist=True)

# Estimate points in the efficient frontier mean - semi standard deviation
ws = port.efficient_frontier(model='Classic', rm=rm, points=20, rf=0, hist=True)

# Estimate the risk parity portfolio for semi standard deviation
w2 = port.rp_optimization(model='Classic', rm=rm, rf=0, b=None, hist=True)

In [ ]:
w2

In [ ]:
stock_indices

In [ ]:
stock_data